In [6]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from waymo_open_dataset.protos import scenario_pb2

# ====== 您的专属配置路径 ====== #
FILENAME = r"/mnt/d/Datasets/WOD_E2E_Camera_v1/val/val_202504211843.tfrecord-00008-of-00093"
SEG_ID   = "fb0ed944efebd34d756103188d59da85"
# ============================== #

def extract_preference_trajectory(tfrecord_path, target_seg_id):
    """从TFRecord中提取preference trajectory (人类驾驶轨迹)"""
    print(f"正在读取文件: {tfrecord_path}")
    dataset = tf.data.TFRecordDataset(tfrecord_path, compression_type='')
    
    for i, data in enumerate(dataset):
        # 解析Scenario protobuf
        scenario = scenario_pb2.Scenario()
        scenario.ParseFromString(bytearray(data.numpy()))
        
        # 匹配目标segment
        if scenario.scenario_id == target_seg_id:
            print(f"\n✅ 找到目标segment: {target_seg_id} (在文件中的第 {i+1} 个scenario)")
            print(f"  - 时间范围: {scenario.timestamps_seconds[0]:.2f}s 到 {scenario.timestamps_seconds[-1]:.2f}s")
            print(f"  - 总帧数: {len(scenario.tracks[scenario.sdc_track_index].states)}")
            
            # 提取自车轨迹 (preference trajectory)
            sdc_track = scenario.tracks[scenario.sdc_track_index]
            trajectory = {
                'center_x': [],
                'center_y': [],
                'heading': [],
                'velocity_x': [],
                'velocity_y': [],
                'valid': [],
                'timestamps': np.array(scenario.timestamps_seconds)  # 直接使用scenario的时间戳
            }
            
            # 收集所有状态
            for state in sdc_track.states:
                trajectory['center_x'].append(state.center_x)
                trajectory['center_y'].append(state.center_y)
                trajectory['heading'].append(state.heading)
                trajectory['velocity_x'].append(state.velocity_x)
                trajectory['velocity_y'].append(state.velocity_y)
                trajectory['valid'].append(state.valid)
            
            # 转换为numpy数组并过滤无效点
            valid_mask = np.array(trajectory['valid'], dtype=bool)
            for key in ['center_x', 'center_y', 'heading', 'velocity_x', 'velocity_y']:
                trajectory[key] = np.array(trajectory[key])[valid_mask]
            
            # 只保留有效点对应的时间戳
            trajectory['timestamps'] = trajectory['timestamps'][valid_mask]
            
            print(f"  - 有效轨迹点: {len(trajectory['center_x'])}")
            return trajectory
    
    raise ValueError(f"❌ 未在文件中找到segment ID: {target_seg_id}\n"
                     f"  请确认:\n"
                     f"  1. SEG_ID是否正确\n"
                     f"  2. 该segment是否确实在此TFRecord文件中\n"
                     f"  3. 是否使用了正确的WOD-E2E版本数据集")

def analyze_trajectory(trajectory):
    """分析偏好轨迹的关键特征"""
    # 1. 计算行驶距离
    positions = np.column_stack((trajectory['center_x'], trajectory['center_y']))
    if len(positions) < 2:
        raise ValueError("轨迹点不足，无法计算距离")
    
    diffs = np.diff(positions, axis=0)
    segment_distances = np.linalg.norm(diffs, axis=1)
    total_distance = np.sum(segment_distances)
    
    # 2. 计算速度 (m/s)
    vx = trajectory['velocity_x']
    vy = trajectory['velocity_y']
    speeds = np.sqrt(vx**2 + vy**2)
    
    # 3. 计算加速度 (m/s²)
    time_diff = np.diff(trajectory['timestamps'])
    if len(time_diff) < 1:
        acc = np.array([])
    else:
        speed_diff = np.diff(speeds)
        acc = speed_diff / np.maximum(time_diff, 1e-5)  # 避免除零
    
    # 4. 计算转向角变化率 (rad/s)
    heading_diff = np.diff(trajectory['heading'])
    if len(heading_diff) > 0:
        # 处理角度环绕问题 (-pi到pi)
        heading_diff = (heading_diff + np.pi) % (2 * np.pi) - np.pi
        steering_rates = heading_diff / np.maximum(time_diff, 1e-5)
    else:
        steering_rates = np.array([])
    
    # 5. 计算轨迹曲率 (1/m)
    curvature = np.array([])
    if len(positions) >= 3:
        dx = np.diff(trajectory['center_x'])
        dy = np.diff(trajectory['center_y'])
        ddx = np.diff(dx) / np.maximum(np.diff(trajectory['timestamps'][:-1]), 1e-5)
        ddy = np.diff(dy) / np.maximum(np.diff(trajectory['timestamps'][:-1]), 1e-5)
        
        # 避免除零
        denominator = (dx[:-1]**2 + dy[:-1]**2)**1.5
        denominator = np.maximum(denominator, 1e-5)
        
        curvature = np.abs(dx[:-1]*ddy - dy[:-1]*ddx) / denominator
    
    return {
        'total_distance': total_distance,
        'avg_speed': np.mean(speeds) if len(speeds) > 0 else 0,
        'max_speed': np.max(speeds) if len(speeds) > 0 else 0,
        'max_acceleration': np.max(np.abs(acc)) if len(acc) > 0 else 0,
        'max_steering_rate': np.max(np.abs(steering_rates)) if len(steering_rates) > 0 else 0,
        'max_curvature': np.max(np.abs(curvature)) if len(curvature) > 0 else 0,
        'speed_profile': speeds,
        'acceleration_profile': acc,
        'steering_profile': steering_rates,
        'curvature_profile': curvature,
        'positions': positions
    }

def visualize_trajectory(trajectory, analysis):
    """可视化轨迹和动态指标"""
    fig, axs = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'Preference Trajectory Analysis: {SEG_ID}', fontsize=16)
    
    # 1. 轨迹俯视图
    axs[0, 0].plot(trajectory['center_x'], trajectory['center_y'], 'b-', linewidth=2, label='Preference Trajectory')
    axs[0, 0].scatter(trajectory['center_x'][0], trajectory['center_y'][0], c='g', s=100, label='Start')
    axs[0, 0].scatter(trajectory['center_x'][-1], trajectory['center_y'][-1], c='r', s=100, label='End')
    axs[0, 0].set_title('Top-down Trajectory')
    axs[0, 0].set_xlabel('X (m)')
    axs[0, 0].set_ylabel('Y (m)')
    axs[0, 0].legend()
    axs[0, 0].grid(True)
    axs[0, 0].axis('equal')
    
    # 2. 速度曲线
    if len(analysis['speed_profile']) > 0:
        axs[0, 1].plot(trajectory['timestamps'][:len(analysis['speed_profile'])], 
                      analysis['speed_profile'], 'g-', linewidth=2)
        axs[0, 1].set_title(f'Speed Profile (Avg: {analysis["avg_speed"]:.2f} m/s)')
    else:
        axs[0, 1].text(0.5, 0.5, 'No speed data', ha='center', va='center')
    axs[0, 1].set_xlabel('Time (s)')
    axs[0, 1].set_ylabel('Speed (m/s)')
    axs[0, 1].grid(True)
    
    # 3. 加速度曲线
    if len(analysis['acceleration_profile']) > 0:
        acc_times = trajectory['timestamps'][1:1+len(analysis['acceleration_profile'])]
        axs[1, 0].plot(acc_times, analysis['acceleration_profile'], 'r-', linewidth=2)
        axs[1, 0].set_title(f'Acceleration (Max: {analysis["max_acceleration"]:.2f} m/s²)')
    else:
        axs[1, 0].text(0.5, 0.5, 'No acceleration data', ha='center', va='center')
    axs[1, 0].set_xlabel('Time (s)')
    axs[1, 0].set_ylabel('Acceleration (m/s²)')
    axs[1, 0].grid(True)
    
    # 4. 曲率热力图
    if len(analysis['curvature_profile']) > 0 and len(analysis['positions']) > 2:
        # 曲率对应的位置是中间点
        curve_positions = analysis['positions'][1:-1]
        sc = axs[1, 1].scatter(curve_positions[:, 0], curve_positions[:, 1], 
                              c=analysis['curvature_profile'], 
                              s=50, cmap='viridis', vmin=0, vmax=0.1)
        axs[1, 1].set_title(f'Trajectory Curvature (Max: {analysis["max_curvature"]:.4f} 1/m)')
        plt.colorbar(sc, ax=axs[1, 1], label='Curvature (1/m)')
    else:
        axs[1, 1].text(0.5, 0.5, 'Insufficient points for curvature', ha='center', va='center')
    
    axs[1, 1].set_xlabel('X (m)')
    axs[1, 1].set_ylabel('Y (m)')
    axs[1, 1].grid(True)
    axs[1, 1].axis('equal')
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])  # 为总标题留出空间
    output_path = f'preference_trajectory_{SEG_ID[:8]}.png'
    plt.savefig(output_path, dpi=150)
    print(f"📊 可视化图表已保存至: {output_path}")
    plt.show()

def animate_trajectory(trajectory):
    """生成轨迹动画"""
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_title(f'Animated Preference Trajectory: {SEG_ID}')
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.grid(True)
    
    # 设置坐标范围 (增加10%边距)
    x_min, x_max = np.min(trajectory['center_x']), np.max(trajectory['center_x'])
    y_min, y_max = np.min(trajectory['center_y']), np.max(trajectory['center_y'])
    margin = 0.1
    x_range = x_max - x_min
    y_range = y_max - y_min
    ax.set_xlim(x_min - margin*x_range, x_max + margin*x_range)
    ax.set_ylim(y_min - margin*y_range, y_max + margin*y_range)
    
    # 绘制完整轨迹
    ax.plot(trajectory['center_x'], trajectory['center_y'], 'b-', alpha=0.3, linewidth=1, label='Full Path')
    
    # 初始化车辆位置
    point, = ax.plot([], [], 'ro', markersize=8, label='Vehicle Position')
    trail, = ax.plot([], [], 'r-', linewidth=2, label='Recent Path')
    time_text = ax.text(0.05, 0.95, '', transform=ax.transAxes, fontsize=12)
    
    # 添加图例
    ax.legend(loc='upper right')
    
    def init():
        point.set_data([], [])
        trail.set_data([], [])
        time_text.set_text('')
        return point, trail, time_text
    
    def animate(i):
        # 更新车辆位置
        point.set_data(trajectory['center_x'][i], trajectory['center_y'][i])
        
        # 更新轨迹尾迹 (最后20个点)
        start_idx = max(0, i-20)
        trail.set_data(trajectory['center_x'][start_idx:i+1], 
                      trajectory['center_y'][start_idx:i+1])
        
        # 更新时间显示
        time_text.set_text(f'Time: {trajectory["timestamps"][i]:.2f} s')
        
        return point, trail, time_text
    
    ani = FuncAnimation(fig, animate, frames=len(trajectory['center_x']),
                        init_func=init, blit=True, interval=50)
    
    # 保存动画
    output_path = f'trajectory_animation_{SEG_ID[:8]}.mp4'
    ani.save(output_path, writer='ffmpeg', fps=20, dpi=100)
    plt.close()
    print(f"🎬 轨迹动画已生成: {output_path}")

# ====== 主执行流程 ====== #
if __name__ == "__main__":
    print("="*60)
    print(f"🎯 开始处理 segment: {SEG_ID}")
    print(f"📁 数据文件: {FILENAME}")
    print("="*60)
    
    # 检查文件是否存在
    if not os.path.exists(FILENAME):
        print(f"❌ 文件不存在! 请检查路径: {FILENAME}")
        print("💡 路径提示:")
        print("  - 在WSL中: /mnt/d/ 对应 Windows 的 D: 盘")
        print("  - 在纯Windows环境: 请使用 'D:\\Datasets\\...' 格式")
        exit(1)
    
    try:
        # 1. 直接使用指定的TFRecord文件提取轨迹
        trajectory = extract_preference_trajectory(FILENAME, SEG_ID)
        
        # 2. 分析轨迹特征
        analysis = analyze_trajectory(trajectory)
        print("\n📊 轨迹分析结果:")
        print(f"- 总行驶距离: {analysis['total_distance']:.2f} 米")
        print(f"- 平均速度: {analysis['avg_speed']:.2f} m/s ({analysis['avg_speed']*3.6:.1f} km/h)")
        print(f"- 最大速度: {analysis['max_speed']:.2f} m/s ({analysis['max_speed']*3.6:.1f} km/h)")
        print(f"- 最大加速度: {analysis['max_acceleration']:.2f} m/s²")
        print(f"- 最大转向速率: {analysis['max_steering_rate']:.4f} rad/s")
        print(f"- 最大曲率: {analysis['max_curvature']:.6f} 1/m (曲率半径: {1/max(1e-6, analysis['max_curvature']):.2f} 米)")
        
        # 3. 可视化结果
        visualize_trajectory(trajectory, analysis)
        
        # 4. 生成轨迹动画 (可选，取消注释启用)
        # print("\n🎬 正在生成轨迹动画 (可能需要1-2分钟)...")
        # animate_trajectory(trajectory)
        
        print("\n✅ 处理完成! 所有结果已保存")
        
    except Exception as e:
        print(f"\n❌ 处理过程中出错: {str(e)}")
        import traceback
        traceback.print_exc()
        print("\n💡 建议检查:")
        print("1. 文件路径是否正确 (特别是Windows路径中的斜杠方向)")
        print("2. 是否安装了正确版本的waymo-open-dataset包")
        print("   推荐安装命令: pip install waymo-open-dataset-tf-2-12-0==1.6.1")
        print("3. 数据集完整性 (尝试用其他segment测试)")
        print("4. 磁盘空间是否充足 (动画生成需要额外空间)")

🎯 开始处理 segment: fb0ed944efebd34d756103188d59da85
📁 数据文件: /mnt/d/Datasets/WOD_E2E_Camera_v1/val/val_202504211843.tfrecord-00008-of-00093
正在读取文件: /mnt/d/Datasets/WOD_E2E_Camera_v1/val/val_202504211843.tfrecord-00008-of-00093

❌ 处理过程中出错: Error parsing message with type 'waymo.open_dataset.Scenario'

💡 建议检查:
1. 文件路径是否正确 (特别是Windows路径中的斜杠方向)
2. 是否安装了正确版本的waymo-open-dataset包
   推荐安装命令: pip install waymo-open-dataset-tf-2-12-0==1.6.1
3. 数据集完整性 (尝试用其他segment测试)
4. 磁盘空间是否充足 (动画生成需要额外空间)


Traceback (most recent call last):
  File "/tmp/ipykernel_34783/4048847090.py", line 263, in <module>
    trajectory = extract_preference_trajectory(FILENAME, SEG_ID)
  File "/tmp/ipykernel_34783/4048847090.py", line 21, in extract_preference_trajectory
    scenario.ParseFromString(bytearray(data.numpy()))
google.protobuf.message.DecodeError: Error parsing message with type 'waymo.open_dataset.Scenario'


找到 rater key moment

In [69]:
import os, glob
import tensorflow as tf

def list_tfrecords(folder):
    files = sorted(glob.glob(os.path.join(folder, "*.tfrecord*")))
    if not files:
        raise FileNotFoundError(f"No tfrecord files under: {folder}")
    return files

def parse_e2ed(raw_bytes: bytes):
    m = wod_e2ed_pb2.E2EDFrame()
    m.ParseFromString(raw_bytes)
    return m

def split_context_name(ctx_name: str):
    if not ctx_name or "-" not in ctx_name:
        return ctx_name, None
    base, suf = ctx_name.rsplit("-", 1)
    try:
        return base, int(suf)
    except ValueError:
        return ctx_name, None

def has_valid_pref_scores(e2e):
    if len(e2e.preference_trajectories) == 0:
        return False
    # 任意一个 score >= 0 就算“有标注”
    for p in e2e.preference_trajectories:
        if hasattr(p, "preference_score"):
            try:
                if float(p.preference_score) >= 0:
                    return True
            except Exception:
                pass
    return False

def find_rater_keyframe_all_val(val_dir, target_seg_id):
    """
    扫全 val，找到该 segment 的 rater keyframe（score>=0）
    返回: (e2e, file_path, record_idx, key_idx, scores_list)
    找不到返回 None
    """
    for fp in list_tfrecords(val_dir):
        ds = tf.data.TFRecordDataset(fp, compression_type=("GZIP" if fp.endswith(".gz") else ""))
        for i, r in enumerate(ds):
            e2e = parse_e2ed(r.numpy())
            ctx = e2e.frame.context.name
            seg_id, key_idx = split_context_name(ctx)
            if seg_id != target_seg_id:
                continue

            if has_valid_pref_scores(e2e):
                scores = []
                for p in e2e.preference_trajectories:
                    scores.append(float(getattr(p, "preference_score", -1.0)))
                return e2e, fp, i, key_idx, scores

    return None


In [70]:
VAL_DIR = r"/mnt/d/Datasets/WOD_E2E_Camera_v1/val"
SEG_ID  = "fb0ed944efebd34d756103188d59da85"

res = find_rater_keyframe_all_val(VAL_DIR, SEG_ID)
print("found:", res is not None)
if res:
    e2e, fp, ridx, key_idx, scores = res
    print("file:", fp)
    print("record:", ridx)
    print("context.name:", e2e.frame.context.name)
    print("key_idx:", key_idx)
    print("scores:", scores)


found: True
file: /mnt/d/Datasets/WOD_E2E_Camera_v1/val/val_202504211843.tfrecord-00026-of-00093
record: 373
context.name: fb0ed944efebd34d756103188d59da85-147
key_idx: 147
scores: [9.0, 6.0, 10.0]


In [19]:
from google.protobuf.descriptor import FieldDescriptor

def pb_set_fields(msg, title=""):
    if title:
        print(f"\n=== {title} (set fields) ===")
    for fd, val in msg.ListFields():
        if fd.label == FieldDescriptor.LABEL_REPEATED:
            print(f"- {fd.name}: repeated len={len(val)}")
        else:
            if fd.type == FieldDescriptor.TYPE_MESSAGE:
                print(f"- {fd.name}: message<{fd.message_type.name}> subfields={len(val.ListFields())}")
            else:
                print(f"- {fd.name}: {val}")

def pb_defined_fields(msg, title=""):
    if title:
        print(f"\n=== {title} (all defined fields) ===")
    for fd in msg.DESCRIPTOR.fields:
        rep = "repeated" if fd.label == FieldDescriptor.LABEL_REPEATED else "scalar"
        print(f"- {fd.name}: {rep}, type={fd.type}")

print("num preference_trajectories:", len(e2e.preference_trajectories))
pb_set_fields(e2e.preference_trajectories[0], "preference_trajectories[0]")
pb_defined_fields(e2e.preference_trajectories[0], "preference_trajectories[0]")


num preference_trajectories: 3

=== preference_trajectories[0] (set fields) ===
- pos_x: repeated len=21
- pos_y: repeated len=21
- preference_score: 9.0

=== preference_trajectories[0] (all defined fields) ===
- pos_x: repeated, type=2
- pos_y: repeated, type=2
- pos_z: repeated, type=2
- vel_x: repeated, type=2
- vel_y: repeated, type=2
- accel_x: repeated, type=2
- accel_y: repeated, type=2
- preference_score: scalar, type=2


In [20]:
import numpy as np
from google.protobuf.descriptor import FieldDescriptor

def pick_repeated_message_field(msg, prefer_kws=("traj","trajectory","point","points","waypoint","waypoints","sample","samples","states","state")):
    best = None
    for fd, val in msg.ListFields():
        if fd.label != FieldDescriptor.LABEL_REPEATED:
            continue
        if fd.type != FieldDescriptor.TYPE_MESSAGE:
            continue
        score = sum(k in fd.name.lower() for k in prefer_kws)
        cand = (score, fd.name, val)
        if best is None or cand[0] > best[0]:
            best = cand
    if best is None:
        return None, None
    return best[1], best[2]

def extract_xy_from_point_msg_list(msg_list):
    if len(msg_list) == 0:
        return None

    p0 = msg_list[0]
    # 常见命名：x/y 或 pos_x/pos_y
    x_name = "x" if hasattr(p0, "x") else ("pos_x" if hasattr(p0, "pos_x") else None)
    y_name = "y" if hasattr(p0, "y") else ("pos_y" if hasattr(p0, "pos_y") else None)
    z_name = "z" if hasattr(p0, "z") else ("pos_z" if hasattr(p0, "pos_z") else None)

    if x_name and y_name:
        x = np.array([float(getattr(p, x_name)) for p in msg_list], dtype=np.float32)
        y = np.array([float(getattr(p, y_name)) for p in msg_list], dtype=np.float32)
        z = np.array([float(getattr(p, z_name)) for p in msg_list], dtype=np.float32) if z_name else None
        return {"x": x, "y": y, "z": z, "__fields__": (x_name, y_name, z_name)}
    else:
        # 抽不到，就返回字段预览
        return {"__preview__": [fd.name for fd, _ in p0.ListFields()]}

def extract_preference_trajectories(e2e):
    out = []
    for i, p in enumerate(e2e.preference_trajectories):
        score = float(getattr(p, "preference_score", np.nan))
        item = {"i": i, "score": score, "traj": None, "meta": {}}

        fname, msg_list = pick_repeated_message_field(p)
        item["meta"]["points_field"] = fname

        if fname is not None and len(msg_list) > 0:
            item["traj"] = extract_xy_from_point_msg_list(msg_list)
        else:
            # 兜底：看看是否是 repeated scalar fields（少见但可能）
            # 例如 pos_x/pos_y 在 p 自身就是 repeated scalar
            scalar = {}
            for k in ("pos_x","pos_y","pos_z","x","y","z"):
                if hasattr(p, k):
                    arr = np.array(list(getattr(p, k)), dtype=np.float32)
                    if arr.size > 0:
                        scalar[k] = arr
            if scalar:
                item["traj"] = scalar
                item["meta"]["points_field"] = "__repeated_scalar_fields__"

        out.append(item)
    return out

prefs = extract_preference_trajectories(e2e)
for it in prefs:
    print(it["i"], "score=", it["score"], "points_field=", it["meta"].get("points_field"))
    if isinstance(it["traj"], dict) and "x" in it["traj"]:
        print("  traj_len=", len(it["traj"]["x"]), "xy_fields=", it["traj"].get("__fields__"))
    elif isinstance(it["traj"], dict) and "__preview__" in it["traj"]:
        print("  point fields preview:", it["traj"]["__preview__"])
    elif isinstance(it["traj"], dict):
        print("  scalar keys:", list(it["traj"].keys())[:10])


0 score= 9.0 points_field= __repeated_scalar_fields__
  scalar keys: ['pos_x', 'pos_y']
1 score= 6.0 points_field= __repeated_scalar_fields__
  scalar keys: ['pos_x', 'pos_y']
2 score= 10.0 points_field= __repeated_scalar_fields__
  scalar keys: ['pos_x', 'pos_y']


In [21]:
import matplotlib.pyplot as plt

def plot_preferences(prefs):
    plt.figure(figsize=(6,6))
    ok_any = False
    for it in prefs:
        t = it["traj"]
        if not isinstance(t, dict):
            continue

        if "x" in t and "y" in t and t["x"] is not None:
            x, y = t["x"], t["y"]
        elif "pos_x" in t and "pos_y" in t:
            x, y = t["pos_x"], t["pos_y"]
        else:
            continue

        ok_any = True
        plt.plot(x, y, marker="o", linewidth=2, label=f"pref{it['i']} score={it['score']:.0f}")

    if not ok_any:
        print("⚠️ 没抽到可绘制的 (x,y) 轨迹点；请看 Block 1 的字段结构输出，调整字段名。")
        return

    plt.axhline(0, linewidth=1)
    plt.axvline(0, linewidth=1)
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlabel("x (forward, vehicle frame)")
    plt.ylabel("y (left, vehicle frame)")
    plt.title("Preference trajectories @ key_idx=147")
    plt.legend()
    plt.show()

plot_preferences(prefs)


In [22]:
import json
def save_prefs_json(prefs, out_path):
    serial = []
    for it in prefs:
        d = {"i": it["i"], "score": float(it["score"]), "meta": it["meta"]}
        t = it["traj"]
        if isinstance(t, dict) and "x" in t:
            d["x"] = it["traj"]["x"].tolist()
            d["y"] = it["traj"]["y"].tolist()
            d["z"] = it["traj"]["z"].tolist() if it["traj"].get("z") is not None else None
        serial.append(d)
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(serial, f, ensure_ascii=False, indent=2)
    print("saved:", out_path)

save_prefs_json(prefs, f"{SEG_ID}_key{key_idx}_prefs.json")


saved: fb0ed944efebd34d756103188d59da85_key147_prefs.json


In [23]:
import numpy as np

def get_ego_future_xy(e2e):
    fut = getattr(e2e, "future_states", None)
    if fut is None:
        return None, None

    # 你目前的数据里常见字段
    if hasattr(fut, "pos_x") and hasattr(fut, "pos_y"):
        x = np.array(list(fut.pos_x), dtype=np.float32)
        y = np.array(list(fut.pos_y), dtype=np.float32)
        return x, y

    # 兜底（如果将来字段名变化）
    if hasattr(fut, "x") and hasattr(fut, "y"):
        x = np.array(list(fut.x), dtype=np.float32)
        y = np.array(list(fut.y), dtype=np.float32)
        return x, y

    return None, None

x_fut, y_fut = get_ego_future_xy(e2e)
print("ego future:", None if x_fut is None else (len(x_fut), len(y_fut)))


ego future: (20, 20)


In [24]:
import matplotlib.pyplot as plt

def _pref_xy(it):
    """从 prefs 的一个元素里取出 (x,y)"""
    t = it.get("traj", None)
    if not isinstance(t, dict):
        return None, None

    # 你之前的抽取可能是 x/y
    if "x" in t and "y" in t and t["x"] is not None and t["y"] is not None:
        return np.asarray(t["x"], dtype=np.float32), np.asarray(t["y"], dtype=np.float32)

    # 或者 repeated scalar fields pos_x/pos_y
    if "pos_x" in t and "pos_y" in t:
        return np.asarray(t["pos_x"], dtype=np.float32), np.asarray(t["pos_y"], dtype=np.float32)

    return None, None

def plot_pref_and_ego_future(prefs, ego_xy, title=None):
    x_fut, y_fut = ego_xy

    plt.figure(figsize=(7, 7))

    # 1) preference trajectories（3条）
    any_pref = False
    for it in prefs:
        x, y = _pref_xy(it)
        if x is None:
            continue
        any_pref = True
        score = it.get("score", None)
        plt.plot(x, y, marker="o", linewidth=2, label=f"pref{it.get('i','?')} score={score}")

    # 2) ego future（20点）
    if x_fut is not None and y_fut is not None:
        plt.plot(x_fut, y_fut, marker="x", linewidth=3, label=f"ego future ({len(x_fut)} pts)")
    else:
        print("⚠️ ego future (x,y) not found in future_states")

    if not any_pref:
        print("⚠️ preference trajectories 没抽到可绘制的 (x,y)，请先检查 prefs 的 traj 结构。")

    plt.axhline(0, linewidth=1)
    plt.axvline(0, linewidth=1)
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlabel("x (forward, vehicle frame)")
    plt.ylabel("y (left, vehicle frame)")
    plt.title(title or "Preference trajectories vs Ego future (vehicle frame)")
    plt.legend()
    plt.show()

plot_pref_and_ego_future(
    prefs,
    (x_fut, y_fut),
    title=f"{SEG_ID} @ key_idx={key_idx}"
)


In [25]:
def save_pref_and_ego_plot(prefs, ego_xy, out_png, title=None):
    x_fut, y_fut = ego_xy
    plt.figure(figsize=(7, 7))

    for it in prefs:
        x, y = _pref_xy(it)
        if x is None:
            continue
        plt.plot(x, y, marker="o", linewidth=2, label=f"pref{it.get('i','?')} score={it.get('score',None)}")

    if x_fut is not None and y_fut is not None:
        plt.plot(x_fut, y_fut, marker="x", linewidth=3, label=f"ego future ({len(x_fut)} pts)")

    plt.axhline(0, linewidth=1)
    plt.axvline(0, linewidth=1)
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlabel("x (forward)")
    plt.ylabel("y (left)")
    plt.title(title or "Preference vs Ego future")
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_png, dpi=200)
    plt.close()
    print("saved:", out_png)

save_pref_and_ego_plot(prefs, (x_fut, y_fut), f"{SEG_ID}_key{key_idx}_pref_vs_ego_future.png",
                       title=f"{SEG_ID} @ key_idx={key_idx}")


saved: fb0ed944efebd34d756103188d59da85_key147_pref_vs_ego_future.png


In [26]:
print("context.name:", e2e.frame.context.name)
print("num images:", len(e2e.frame.images))
print("image bytes lens:", [len(img.image) for img in e2e.frame.images])


context.name: fb0ed944efebd34d756103188d59da85-147
num images: 8
image bytes lens: [229572, 180035, 269647, 198310, 267741, 219866, 187987, 172591]


In [27]:
import os

def export_keymoment_jpegs(e2e, out_dir, prefix=None):
    os.makedirs(out_dir, exist_ok=True)

    ctx = e2e.frame.context.name
    prefix = prefix or ctx.replace("/", "_")

    saved = []
    for img in e2e.frame.images:
        cam_id = int(img.name)  # 你现在环境里 cam 用数字（1~8）
        fn = f"{prefix}_cam{cam_id}.jpg"
        fp = os.path.join(out_dir, fn)

        # JPEG bytes 直接写文件（最快）
        with open(fp, "wb") as f:
            f.write(img.image)

        saved.append((cam_id, fp, len(img.image)))

    saved.sort(key=lambda x: x[0])
    return saved

OUT_DIR = r"/mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_jpegs"  # 你也可以换成 Windows 路径
saved = export_keymoment_jpegs(e2e, OUT_DIR, prefix=f"{SEG_ID}_key{key_idx}")
print("saved", len(saved), "files:")
for cam_id, fp, nbytes in saved:
    print(f"  cam{cam_id}: {nbytes} bytes -> {fp}")


saved 8 files:
  cam1: 180035 bytes -> /mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_jpegs/fb0ed944efebd34d756103188d59da85_key147_cam1.jpg
  cam2: 229572 bytes -> /mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_jpegs/fb0ed944efebd34d756103188d59da85_key147_cam2.jpg
  cam3: 269647 bytes -> /mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_jpegs/fb0ed944efebd34d756103188d59da85_key147_cam3.jpg
  cam4: 198310 bytes -> /mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_jpegs/fb0ed944efebd34d756103188d59da85_key147_cam4.jpg
  cam5: 267741 bytes -> /mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_jpegs/fb0ed944efebd34d756103188d59da85_key147_cam5.jpg
  cam6: 172591 bytes -> /mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_jpegs/fb0ed944efebd34d756103188d59da85_key147_cam6.jpg
  cam7: 187987 bytes -> /mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_jpegs/fb0ed944efebd34d756103188d59da85_key147_cam7.jpg
  cam8: 219866 bytes -> /mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_jpegs/fb0ed944efebd34d756103188d59da85_key147_ca

In [29]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

def decode_images_for_preview(e2e):
    imgs, labels = [], []
    for img in e2e.frame.images:
        cam_id = int(img.name)
        arr = tf.io.decode_jpeg(img.image).numpy()
        imgs.append(arr)
        labels.append(f"cam{cam_id} {arr.shape[1]}x{arr.shape[0]}")
    return imgs, labels

def show_grid(imgs, labels, cols=4, figsize=(14, 8)):
    n = len(imgs)
    rows = int(np.ceil(n / cols))
    plt.figure(figsize=figsize)
    for i in range(n):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(imgs[i])
        plt.title(labels[i], fontsize=10)
        plt.axis("off")
    plt.tight_layout()
    plt.show()

imgs, labels = decode_images_for_preview(e2e)
show_grid(imgs, labels, cols=4)


In [30]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

def get_image_by_cam_id(frame, cam_id: int):
    """返回 (rgb, image_msg)；找不到返回 (None, None)。"""
    cam_id = int(cam_id)
    for im in frame.images:
        if int(im.name) == cam_id:
            rgb = tf.io.decode_jpeg(im.image).numpy()
            return rgb, im
    return None, None

def get_camera_calib_by_cam_id(frame, cam_id: int):
    """从 frame.context.camera_calibrations 里取指定相机标定；找不到返回 None。"""
    cam_id = int(cam_id)
    for c in frame.context.camera_calibrations:
        if int(c.name) == cam_id:
            return c
    return None

# 你要画的相机：前视 cam_id=1（你之前也确认 CAM="1" 可用）
CAM_ID = 1
frame = e2e.frame

rgb, img_msg = get_image_by_cam_id(frame, CAM_ID)
calib = get_camera_calib_by_cam_id(frame, CAM_ID)

print("rgb:", None if rgb is None else rgb.shape)
print("calib found:", calib is not None)
if calib is not None:
    intr = list(calib.intrinsic)
    print("intrinsic[0:4] fx,fy,cx,cy =", intr[:4])
    print("extrinsic len =", len(list(calib.extrinsic.transform)))


rgb: (1079, 972, 3)
calib found: True
intrinsic[0:4] fx,fy,cx,cy = [1114.0047607421875, 1113.86767578125, 488.12847900390625, 719.156005859375]
extrinsic len = 16


In [31]:
def calib_to_mats(calib):
    """返回 fx,fy,cx,cy, T_vehicle_from_cam(4x4)."""
    intr = list(calib.intrinsic)
    fx, fy, cx, cy = intr[0], intr[1], intr[2], intr[3]
    T_v_from_c = np.array(list(calib.extrinsic.transform), dtype=np.float32).reshape(4, 4)  # camera->vehicle
    return fx, fy, cx, cy, T_v_from_c

def vehicle_xyz_to_cam_xyz(xyz_vehicle, T_vehicle_from_cam):
    """
    xyz_vehicle: (N,3) in vehicle FLU
    T_vehicle_from_cam: camera->vehicle
    return xyz_cam: (N,3) in camera frame
    """
    xyz_vehicle = np.asarray(xyz_vehicle, dtype=np.float32)
    N = xyz_vehicle.shape[0]
    ones = np.ones((N, 1), dtype=np.float32)
    P_v = np.concatenate([xyz_vehicle, ones], axis=1)  # (N,4)

    T_c_from_v = np.linalg.inv(T_vehicle_from_cam)     # vehicle->camera
    P_c = (T_c_from_v @ P_v.T).T                       # (N,4)
    return P_c[:, :3]

def project_cam_xyz_to_uv(xyz_cam, fx, fy, cx, cy, min_depth=0.5):
    """
    Waymo camera frame: x is depth (forward out of lens), z up, y left. :contentReference[oaicite:3]{index=3}
    输出：
      uv: (M,2) float32
      mask: (N,) bool  表示哪些点投影有效（深度>min_depth）
    """
    xyz_cam = np.asarray(xyz_cam, dtype=np.float32)
    x = xyz_cam[:, 0]  # depth
    y = xyz_cam[:, 1]
    z = xyz_cam[:, 2]
    mask = x > float(min_depth)

    u = cx - fx * (y / np.clip(x, 1e-6, None))
    v = cy - fy * (z / np.clip(x, 1e-6, None))
    uv = np.stack([u, v], axis=1).astype(np.float32)
    return uv, mask

def project_vehicle_xyz_to_uv(xyz_vehicle, calib, min_depth=0.5):
    fx, fy, cx, cy, T_v_from_c = calib_to_mats(calib)
    xyz_cam = vehicle_xyz_to_cam_xyz(xyz_vehicle, T_v_from_c)
    uv, mask = project_cam_xyz_to_uv(xyz_cam, fx, fy, cx, cy, min_depth=min_depth)
    return uv, mask


In [32]:
def ego_future_xyz(e2e):
    fs = e2e.future_states
    # 兼容：有的版本是 repeated scalar fields
    x = np.array(list(getattr(fs, "pos_x", [])), dtype=np.float32)
    y = np.array(list(getattr(fs, "pos_y", [])), dtype=np.float32)
    z = np.array(list(getattr(fs, "pos_z", [])), dtype=np.float32)

    if len(x) == 0 or len(y) == 0:
        raise ValueError("future_states 里没找到 pos_x/pos_y（结构可能不同，需要再打印 fs.ListFields()）")

    if len(z) == 0:
        z = np.zeros_like(x)

    n = min(len(x), len(y), len(z))
    return np.stack([x[:n], y[:n], z[:n]], axis=1)

ego_fut = ego_future_xyz(e2e)
print("ego_future:", ego_fut.shape, "first:", ego_fut[0])


ego_future: (20, 3) first: [ 2.7294922e+00  4.8828125e-03 -2.4108887e-03]


In [33]:
from google.protobuf.descriptor import FieldDescriptor

def _is_message(x):
    return hasattr(x, "ListFields") and callable(getattr(x, "ListFields"))

def _try_extract_pos_arrays_from_message(msg):
    """
    在 msg 这一层尝试直接找 repeated scalar: pos_x/pos_y/(pos_z)
    找到就返回 (N,3)；否则返回 None
    """
    if not _is_message(msg):
        return None

    field_map = {fd.name: val for fd, val in msg.ListFields()}

    # 常见命名：pos_x/pos_y/pos_z
    if "pos_x" in field_map and "pos_y" in field_map:
        x = np.array(list(field_map["pos_x"]), dtype=np.float32)
        y = np.array(list(field_map["pos_y"]), dtype=np.float32)
        z = np.array(list(field_map.get("pos_z", [])), dtype=np.float32)
        if len(z) == 0:
            z = np.zeros_like(x)
        n = min(len(x), len(y), len(z))
        return np.stack([x[:n], y[:n], z[:n]], axis=1)

    # 备选命名：x/y/z（有些 message 会这么叫）
    if "x" in field_map and "y" in field_map:
        x = np.array(list(field_map["x"]), dtype=np.float32)
        y = np.array(list(field_map["y"]), dtype=np.float32)
        z = np.array(list(field_map.get("z", [])), dtype=np.float32)
        if len(z) == 0:
            z = np.zeros_like(x)
        n = min(len(x), len(y), len(z))
        return np.stack([x[:n], y[:n], z[:n]], axis=1)

    return None

def extract_xyz_recursive(msg, max_depth=4):
    """
    递归在 msg 及其子 message 里找 pos_x/pos_y/pos_z（或 x/y/z）
    找到第一条就返回 (N,3)，否则 None
    """
    if max_depth < 0 or (not _is_message(msg)):
        return None

    direct = _try_extract_pos_arrays_from_message(msg)
    if direct is not None and direct.shape[0] > 0:
        return direct

    # 继续往下找：message / repeated message
    for fd, val in msg.ListFields():
        if fd.type != FieldDescriptor.TYPE_MESSAGE:
            continue

        if fd.label == FieldDescriptor.LABEL_REPEATED:
            # repeated message
            if len(val) == 0:
                continue
            cand = extract_xyz_recursive(val[0], max_depth=max_depth - 1)
            if cand is not None:
                return cand
        else:
            cand = extract_xyz_recursive(val, max_depth=max_depth - 1)
            if cand is not None:
                return cand

    return None

def get_preference_trajs_xyz(e2e):
    out = []
    for i, pref in enumerate(e2e.preference_trajectories):
        xyz = extract_xyz_recursive(pref, max_depth=5)
        score = getattr(pref, "preference_score", None)
        print(f"[pref {i}] score={score}, xyz={'None' if xyz is None else xyz.shape}")
        out.append(xyz)
    return out

pref_xyz_list = get_preference_trajs_xyz(e2e)


[pref 0] score=9.0, xyz=(21, 3)
[pref 1] score=6.0, xyz=(21, 3)
[pref 2] score=10.0, xyz=(21, 3)


In [34]:
def draw_polyline(ax, uv, mask, label, linewidth=2):
    uv2 = uv[mask]
    if len(uv2) < 2:
        return
    ax.plot(uv2[:, 0], uv2[:, 1], linewidth=linewidth, label=label)

def overlay_trajectories_on_front(rgb, calib, ego_future_xyz, pref_xyz_list, title="front overlay"):
    H, W = rgb.shape[0], rgb.shape[1]

    fig = plt.figure(figsize=(14, 8))
    ax = plt.gca()
    ax.imshow(rgb)
    ax.set_title(title)
    ax.axis("off")

    # ego future（第4条）
    uv_e, m_e = project_vehicle_xyz_to_uv(ego_future_xyz, calib, min_depth=0.5)
    # 再加一层：过滤掉出界点
    m_e = m_e & (uv_e[:,0] >= 0) & (uv_e[:,0] < W) & (uv_e[:,1] >= 0) & (uv_e[:,1] < H)
    draw_polyline(ax, uv_e, m_e, label="ego_future(20)")

    # 3 条 preference
    for i, xyz in enumerate(pref_xyz_list):
        if xyz is None or len(xyz) == 0:
            continue
        uv, m = project_vehicle_xyz_to_uv(xyz, calib, min_depth=0.5)
        m = m & (uv[:,0] >= 0) & (uv[:,0] < W) & (uv[:,1] >= 0) & (uv[:,1] < H)
        draw_polyline(ax, uv, m, label=f"pref_{i}")

    ax.legend()
    plt.show()

overlay_trajectories_on_front(
    rgb=rgb,
    calib=calib,
    ego_future_xyz=ego_fut,
    pref_xyz_list=pref_xyz_list,
    title=f"SEG key-moment overlay | cam_id={CAM_ID}"
)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def ego_past_xyz(e2e):
    ps = e2e.past_states
    x = np.array(list(getattr(ps, "pos_x", [])), dtype=np.float32)
    y = np.array(list(getattr(ps, "pos_y", [])), dtype=np.float32)
    z = np.array(list(getattr(ps, "pos_z", [])), dtype=np.float32)

    if len(x) == 0 or len(y) == 0:
        raise ValueError("past_states 里没找到 pos_x/pos_y（结构可能不同，需要再打印 ps.ListFields()）")

    if len(z) == 0:
        z = np.zeros_like(x)

    n = min(len(x), len(y), len(z))
    return np.stack([x[:n], y[:n], z[:n]], axis=1)

ego_past = ego_past_xyz(e2e)
print("ego_past:", ego_past.shape)

def _in_bounds_mask(uv, H, W):
    return (uv[:,0] >= 0) & (uv[:,0] < W) & (uv[:,1] >= 0) & (uv[:,1] < H)

def overlay_all_on_front(rgb, calib, ego_past_xyz, ego_future_xyz, pref_xyz_list, title="overlay"):
    H, W = rgb.shape[0], rgb.shape[1]
    fig = plt.figure(figsize=(14, 8))
    ax = plt.gca()
    ax.imshow(rgb)
    ax.set_title(title)
    ax.axis("off")

    # keyframe 点：vehicle frame 原点 (0,0,0)
    origin = np.array([[0.0, 0.0, 0.0]], dtype=np.float32)
    uv0, m0 = project_vehicle_xyz_to_uv(origin, calib, min_depth=0.05)
    if m0[0] and _in_bounds_mask(uv0, H, W)[0]:
        ax.scatter([uv0[0,0]], [uv0[0,1]], s=80, marker="*", label="keyframe (0,0,0)")

    # ego past
    uv_p, mp = project_vehicle_xyz_to_uv(ego_past_xyz, calib, min_depth=0.05)
    mp = mp & _in_bounds_mask(uv_p, H, W)
    if mp.any():
        ax.plot(uv_p[mp,0], uv_p[mp,1], marker=".", linewidth=2, label=f"ego_past({mp.sum()}/{len(mp)})")

    # ego future
    uv_f, mf = project_vehicle_xyz_to_uv(ego_future_xyz, calib, min_depth=0.05)
    mf = mf & _in_bounds_mask(uv_f, H, W)
    if mf.any():
        ax.plot(uv_f[mf,0], uv_f[mf,1], marker="x", linewidth=3, label=f"ego_future({mf.sum()}/{len(mf)})")

    # 3 条 preference
    for i, xyz in enumerate(pref_xyz_list):
        if xyz is None or len(xyz) == 0:
            continue
        uv, m = project_vehicle_xyz_to_uv(xyz, calib, min_depth=0.05)
        m = m & _in_bounds_mask(uv, H, W)
        if m.any():
            score = float(getattr(e2e.preference_trajectories[i], "preference_score", np.nan))
            ax.plot(uv[m,0], uv[m,1], marker="o", linewidth=2, label=f"pref{i} score={score:.0f} ({m.sum()})")

    ax.legend()
    plt.show()

overlay_all_on_front(
    rgb=rgb,
    calib=calib,
    ego_past_xyz=ego_past,
    ego_future_xyz=ego_fut,
    pref_xyz_list=pref_xyz_list,
    title=f"{SEG_ID} key_idx={key_idx} | FRONT(cam_id=1) overlay"
)


ego_past: (16, 3)


In [50]:
def get_image_by_cam_id(frame, cam_id: int):
    cam_id = int(cam_id)
    for im in frame.images:
        if int(im.name) == cam_id:
            rgb = tf.io.decode_jpeg(im.image).numpy()
            return rgb, im
    return None, None

def get_camera_calib_by_cam_id(frame, cam_id: int):
    cam_id = int(cam_id)
    for c in frame.context.camera_calibrations:
        if int(c.name) == cam_id:
            return c
    return None

def overlay_on_cam_id(e2e, cam_id, out_png=None):
    frame = e2e.frame
    rgb, _ = get_image_by_cam_id(frame, cam_id)
    calib = get_camera_calib_by_cam_id(frame, cam_id)
    if rgb is None or calib is None:
        print(f"cam{cam_id}: missing rgb or calib")
        return

    H, W = rgb.shape[0], rgb.shape[1]

    # 投影可见性统计
    def count_visible(xyz):
        uv, m = project_vehicle_xyz_to_uv(xyz, calib, min_depth=0.05)
        m = m & _in_bounds_mask(uv, H, W)
        return int(m.sum())

    v_p = count_visible(ego_past)
    v_f = count_visible(ego_fut)
    v_pref = [count_visible(x) if x is not None else 0 for x in pref_xyz_list]
    print(f"cam{cam_id}: visible past={v_p}, future={v_f}, prefs={v_pref}")

    # 画图
    plt.figure(figsize=(14, 8))
    ax = plt.gca()
    ax.imshow(rgb)
    ax.set_title(f"cam{cam_id} overlay | past={v_p}, future={v_f}, prefs={v_pref}")
    ax.axis("off")

    # keyframe
    origin = np.array([[0.0,0.0,0.0]], dtype=np.float32)
    uv0, m0 = project_vehicle_xyz_to_uv(origin, calib, min_depth=0.05)
    if m0[0] and _in_bounds_mask(uv0, H, W)[0]:
        ax.scatter([uv0[0,0]], [uv0[0,1]], s=80, marker="*", label="keyframe")

    # past/future
    for name, xyz, mk, lw in [
        ("ego_past", ego_past, ".", 2),
        ("ego_future", ego_fut, "x", 3),
    ]:
        uv, m = project_vehicle_xyz_to_uv(xyz, calib, min_depth=0.05)
        m = m & _in_bounds_mask(uv, H, W)
        if m.any():
            ax.plot(uv[m,0], uv[m,1], marker=mk, linewidth=lw, label=name)

    # prefs
    for i, xyz in enumerate(pref_xyz_list):
        if xyz is None or len(xyz) == 0:
            continue
        uv, m = project_vehicle_xyz_to_uv(xyz, calib, min_depth=0.05)
        m = m & _in_bounds_mask(uv, H, W)
        if m.any():
            score = float(getattr(e2e.preference_trajectories[i], "preference_score", np.nan))
            ax.plot(uv[m,0], uv[m,1], marker="o", linewidth=2, label=f"pref{i} s={score:.0f}")

    ax.legend()
    plt.tight_layout()

    if out_png:
        plt.savefig(out_png, dpi=200)
        plt.close()
        print("saved:", out_png)
    else:
        plt.show()

# 对当前 key moment 的 8 路相机都跑一遍，并保存到文件夹（可选）
OUT_DIR = r"/mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_overlays"
os.makedirs(OUT_DIR, exist_ok=True)

for cam_id in sorted([int(im.name) for im in e2e.frame.images]):
    out_png = os.path.join(OUT_DIR, f"{SEG_ID}_key{key_idx}_cam{cam_id}_overlay.png")
    overlay_on_cam_id(e2e, cam_id, out_png=out_png)


cam1: visible past=0, future=18, prefs=[19, 19, 19]


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


saved: /mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_overlays/fb0ed944efebd34d756103188d59da85_key147_cam1_overlay.png
cam2: visible past=0, future=0, prefs=[0, 0, 0]


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


saved: /mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_overlays/fb0ed944efebd34d756103188d59da85_key147_cam2_overlay.png
cam3: visible past=0, future=0, prefs=[0, 0, 0]


...

saved: /mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_overlays/fb0ed944efebd34d756103188d59da85_key147_cam4_overlay.png
cam5: visible past=0, future=0, prefs=[0, 0, 0]


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


saved: /mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_overlays/fb0ed944efebd34d756103188d59da85_key147_cam5_overlay.png
cam6: visible past=0, future=0, prefs=[0, 0, 0]
saved: /mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_overlays/fb0ed944efebd34d756103188d59da85_key147_cam6_overlay.png
cam7: visible past=13, future=0, prefs=[0, 0, 0]


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


saved: /mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_overlays/fb0ed944efebd34d756103188d59da85_key147_cam7_overlay.png
cam8: visible past=0, future=0, prefs=[0, 0, 0]
saved: /mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_overlays/fb0ed944efebd34d756103188d59da85_key147_cam8_overlay.png


In [38]:
import json
import numpy as np
import os

def tolist_safe(x):
    return None if x is None else np.asarray(x).tolist()

def save_overlay_bundle(e2e, cam_id, ego_past_xyz, ego_future_xyz, pref_xyz_list, out_json):
    frame = e2e.frame
    calib = get_camera_calib_by_cam_id(frame, cam_id)
    assert calib is not None

    bundle = {
        "context_name": frame.context.name,
        "segment_id": split_context_name(frame.context.name)[0],
        "key_idx": split_context_name(frame.context.name)[1],
        "cam_id": int(cam_id),
        "scores": [float(p.preference_score) for p in e2e.preference_trajectories],
        "ego_past_xyz": tolist_safe(ego_past_xyz),
        "ego_future_xyz": tolist_safe(ego_future_xyz),
        "pref_xyz": [tolist_safe(x) for x in pref_xyz_list],
    }

    # 也把投影后的 uv 保存一下
    def proj(xyz):
        if xyz is None or len(xyz) == 0:
            return None
        uv, m = project_vehicle_xyz_to_uv(np.asarray(xyz, np.float32), calib, min_depth=0.05)
        return {"uv": uv.tolist(), "valid": m.astype(bool).tolist()}

    bundle["ego_past_uv"] = proj(ego_past_xyz)
    bundle["ego_future_uv"] = proj(ego_future_xyz)
    bundle["pref_uv"] = [proj(x) for x in pref_xyz_list]

    os.makedirs(os.path.dirname(out_json), exist_ok=True)
    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(bundle, f, ensure_ascii=False, indent=2)
    print("saved:", out_json)

save_overlay_bundle(
    e2e, cam_id=1,
    ego_past_xyz=ego_past,
    ego_future_xyz=ego_fut,
    pref_xyz_list=pref_xyz_list,
    out_json=f"/mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_overlays/{SEG_ID}_key{key_idx}_bundle.json"
)


saved: /mnt/d/Datasets/WOD_E2E_Camera_v1/key_moment_overlays/fb0ed944efebd34d756103188d59da85_key147_bundle.json


运动学特征

In [39]:
import os, json
import numpy as np

def rep_arr(msg, field, dtype=np.float32):
    """读取 repeated scalar 字段 -> np.array；不存在/为空返回 None"""
    if msg is None or (not hasattr(msg, field)):
        return None
    v = getattr(msg, field)
    try:
        a = np.array(list(v), dtype=dtype)
        return a if a.size > 0 else None
    except Exception:
        return None

def stack_xyz(x, y, z=None):
    if x is None or y is None:
        return None
    if z is None:
        z = np.zeros_like(x)
    n = min(len(x), len(y), len(z))
    return np.stack([x[:n], y[:n], z[:n]], axis=1)


In [40]:
def extract_ego_kinematics(e2e):
    past = getattr(e2e, "past_states", None)
    fut  = getattr(e2e, "future_states", None)

    # past
    past_pos = stack_xyz(rep_arr(past,"pos_x"), rep_arr(past,"pos_y"), rep_arr(past,"pos_z"))
    past_vel = stack_xyz(rep_arr(past,"vel_x"), rep_arr(past,"vel_y"), rep_arr(past,"vel_z"))
    past_acc = stack_xyz(rep_arr(past,"accel_x"), rep_arr(past,"accel_y"), rep_arr(past,"accel_z"))

    # future（通常只有 pos）
    fut_pos  = stack_xyz(rep_arr(fut,"pos_x"), rep_arr(fut,"pos_y"), rep_arr(fut,"pos_z"))
    fut_vel  = stack_xyz(rep_arr(fut,"vel_x"), rep_arr(fut,"vel_y"), rep_arr(fut,"vel_z"))  # 多数情况 None
    fut_acc  = stack_xyz(rep_arr(fut,"accel_x"), rep_arr(fut,"accel_y"), rep_arr(fut,"accel_z"))  # 多数情况 None

    return {
        "past": {"pos": past_pos, "vel": past_vel, "acc": past_acc},
        "future": {"pos": fut_pos, "vel": fut_vel, "acc": fut_acc},
    }

ego = extract_ego_kinematics(e2e)
print("past pos:", None if ego["past"]["pos"] is None else ego["past"]["pos"].shape)
print("past vel:", None if ego["past"]["vel"] is None else ego["past"]["vel"].shape)
print("past acc:", None if ego["past"]["acc"] is None else ego["past"]["acc"].shape)
print("future pos:", None if ego["future"]["pos"] is None else ego["future"]["pos"].shape)


past pos: (16, 3)
past vel: (16, 3)
past acc: (16, 3)
future pos: (20, 3)


In [42]:
def extract_preferences(e2e, pref_xyz_list):
    prefs = []
    for i, p in enumerate(e2e.preference_trajectories):
        score = float(getattr(p, "preference_score", np.nan))
        xyz = pref_xyz_list[i] if (pref_xyz_list is not None and i < len(pref_xyz_list)) else None
        prefs.append({"i": i, "score": score, "xyz": xyz})
    return prefs

prefs = extract_preferences(e2e, pref_xyz_list)
print([(p["i"], p["score"], None if p["xyz"] is None else p["xyz"].shape) for p in prefs])


[(0, 9.0, (21, 3)), (1, 6.0, (21, 3)), (2, 10.0, (21, 3))]


In [43]:
import pandas as pd

def xyz_to_df(xyz, prefix=""):
    if xyz is None:
        return None
    df = pd.DataFrame(xyz, columns=[f"{prefix}x", f"{prefix}y", f"{prefix}z"])
    df.insert(0, "t", np.arange(len(df), dtype=int))
    return df

def export_bundle(e2e, ego, prefs, out_root):
    ctx = e2e.frame.context.name
    seg_id, key_idx = split_context_name(ctx)

    out_dir = os.path.join(out_root, f"{seg_id}_key{key_idx}")
    os.makedirs(out_dir, exist_ok=True)

    # 1) summary.json（元信息）
    meta = {
        "context_name": ctx,
        "segment_id": seg_id,
        "key_idx": key_idx,
        "intent": int(getattr(e2e, "intent", -1)),
        "scores": [float(p["score"]) for p in prefs],
        "ego_past_len": None if ego["past"]["pos"] is None else int(len(ego["past"]["pos"])),
        "ego_future_len": None if ego["future"]["pos"] is None else int(len(ego["future"]["pos"])),
        "pref_lens": [None if p["xyz"] is None else int(len(p["xyz"])) for p in prefs],
        "coord_frame": "vehicle (x forward, y left, z up)"
    }
    with open(os.path.join(out_dir, "summary.json"), "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    # 2) ego past/future：带 vel/acc 的话一起写进同一个 CSV
    def write_ego_csv(name, block):
        pos, vel, acc = block["pos"], block["vel"], block["acc"]
        if pos is None:
            return
        df = xyz_to_df(pos, prefix="pos_")
        if vel is not None:
            df[["vel_x","vel_y","vel_z"]] = vel[:len(df)]
        if acc is not None:
            df[["acc_x","acc_y","acc_z"]] = acc[:len(df)]
        df.to_csv(os.path.join(out_dir, name), index=False)

    write_ego_csv("ego_past.csv", ego["past"])
    write_ego_csv("ego_future.csv", ego["future"])

    # 3) preference trajectories：每条一个 CSV
    for p in prefs:
        xyz = p["xyz"]
        if xyz is None:
            continue
        df = xyz_to_df(xyz, prefix="pos_")
        fn = f"pref_{p['i']}_score{int(round(p['score']))}.csv"
        df.to_csv(os.path.join(out_dir, fn), index=False)

    print("✅ Exported to:", out_dir)
    return out_dir

OUT_ROOT = r"/mnt/d/Datasets/WOD_E2E_Camera_v1/exports"
out_dir = export_bundle(e2e, ego, prefs, OUT_ROOT)


✅ Exported to: /mnt/d/Datasets/WOD_E2E_Camera_v1/exports/fb0ed944efebd34d756103188d59da85_key147


In [46]:
import os, json
import numpy as np
import pandas as pd

FPS = 10.0  # 你视频用的帧率/采样率，保持一致

def xyz_to_df_scheme_a(xyz, fps=10.0, kind="future_or_pref", prefix="pos_"):
    """
    kind:
      - "past"          -> t_rel_key_idx = -N..-1
      - "future_or_pref"-> t_rel_key_idx = 0..N-1
    """
    if xyz is None:
        return None
    xyz = np.asarray(xyz, dtype=np.float32)
    n = xyz.shape[0]
    t_idx = np.arange(n, dtype=int)

    if kind == "past":
        t_rel = t_idx - n          # -n .. -1
    else:
        t_rel = t_idx              # 0 .. n-1

    df = pd.DataFrame({
        "t_idx": t_idx,
        "t_sec": t_idx / float(fps),
        "t_rel_key_idx": t_rel,
        "t_rel_key_sec": t_rel / float(fps),
        f"{prefix}x": xyz[:,0],
        f"{prefix}y": xyz[:,1],
        f"{prefix}z": xyz[:,2],
    })
    return df

def export_bundle_scheme_a(e2e, ego, prefs, out_root, fps=10.0):
    # 你之前已有 split_context_name()
    ctx = e2e.frame.context.name
    seg_id, key_idx = split_context_name(ctx)

    out_dir = os.path.join(out_root, f"{seg_id}_key{key_idx}")
    os.makedirs(out_dir, exist_ok=True)

    # summary.json
    meta = {
        "context_name": ctx,
        "segment_id": seg_id,
        "key_idx": key_idx,
        "fps_assumed": float(fps),
        "time_convention": {
            "scheme": "A",
            "past": "t_rel_key_idx = -N..-1",
            "future/pref": "t_rel_key_idx = 0..N-1",
            "t_rel_key_sec": "t_rel_key_idx / fps"
        },
        "intent": int(getattr(e2e, "intent", -1)),
        "scores": [float(p["score"]) for p in prefs],
        "coord_frame": "vehicle (x forward, y left, z up)"
    }
    with open(os.path.join(out_dir, "summary.json"), "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    # ego past/future（pos + vel/acc 追加列）
    def write_ego_csv(fname, block, kind):
        pos, vel, acc = block["pos"], block["vel"], block["acc"]
        if pos is None:
            return
        df = xyz_to_df_scheme_a(pos, fps=fps, kind=kind, prefix="pos_")

        # 追加 vel/acc（如果存在）
        if vel is not None:
            vel = np.asarray(vel, dtype=np.float32)
            df[["vel_x","vel_y","vel_z"]] = vel[:len(df)]
        if acc is not None:
            acc = np.asarray(acc, dtype=np.float32)
            df[["acc_x","acc_y","acc_z"]] = acc[:len(df)]

        df.to_csv(os.path.join(out_dir, fname), index=False)

    write_ego_csv("ego_past.csv", ego["past"], kind="past")
    write_ego_csv("ego_future.csv", ego["future"], kind="future_or_pref")

    # preference trajectories
    for p in prefs:
        xyz = p["xyz"]
        if xyz is None:
            continue
        df = xyz_to_df_scheme_a(xyz, fps=fps, kind="future_or_pref", prefix="pos_")
        fn = f"pref_{p['i']}_score{int(round(p['score']))}.csv"
        df.to_csv(os.path.join(out_dir, fn), index=False)

    print("✅ Exported (scheme A) to:", out_dir)
    return out_dir

OUT_ROOT = r"/mnt/d/Datasets/WOD_E2E_Camera_v1/exports"
out_dir = export_bundle_scheme_a(e2e, ego, prefs, OUT_ROOT, fps=FPS)


✅ Exported (scheme A) to: /mnt/d/Datasets/WOD_E2E_Camera_v1/exports/fb0ed944efebd34d756103188d59da85_key147


In [49]:
import os, json
import numpy as np
import pandas as pd

def wrap_to_pi(a):
    """把角度 wrap 到 (-pi, pi]"""
    return (a + np.pi) % (2*np.pi) - np.pi

def add_heading_speed_to_ego_csv(csv_path, speed_eps=0.2):
    """
    给 ego_past.csv / ego_future.csv 追加:
      speed = sqrt(vx^2 + vy^2)
      heading_vel_rad = atan2(vy, vx) (wrap to pi)
      heading_vel_deg = heading_vel_rad * 180/pi
    低速 speed < speed_eps -> heading 设为 NaN，避免抖动
    """
    if not os.path.exists(csv_path):
        print("skip missing:", csv_path)
        return

    df = pd.read_csv(csv_path)

    # 需要 vel_x/vel_y 才能算 heading_vel
    if ("vel_x" not in df.columns) or ("vel_y" not in df.columns):
        print(f"skip (no vel_x/vel_y): {csv_path}")
        return

    vx = df["vel_x"].to_numpy(dtype=np.float32)
    vy = df["vel_y"].to_numpy(dtype=np.float32)

    speed = np.sqrt(vx*vx + vy*vy)
    heading = np.arctan2(vy, vx)
    heading = wrap_to_pi(heading)

    # 低速屏蔽
    heading = heading.astype(np.float32)
    heading[speed < float(speed_eps)] = np.nan

    df["speed"] = speed
    df["heading_vel_rad"] = heading
    df["heading_vel_deg"] = heading * 180.0 / np.pi

    df.to_csv(csv_path, index=False)
    print("updated:", csv_path)

def update_summary_heading_source(summary_json_path, source="velocity_heading", speed_eps=0.2):
    if not os.path.exists(summary_json_path):
        print("skip missing:", summary_json_path)
        return
    with open(summary_json_path, "r", encoding="utf-8") as f:
        meta = json.load(f)
    meta["heading_source"] = source
    meta["heading_velocity_speed_eps_mps"] = float(speed_eps)  # 这里单位按你数据的速度单位理解（通常 m/s）
    with open(summary_json_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)
    print("updated:", summary_json_path)

# ====== 用法：对你某个样本导出目录执行 ======
# 例如：out_dir = "/mnt/d/Datasets/WOD_E2E_Camera_v1/exports/<seg>_key<idx>"
def augment_export_folder(out_dir, speed_eps=0.2):
    add_heading_speed_to_ego_csv(os.path.join(out_dir, "ego_past.csv"), speed_eps=speed_eps)
    add_heading_speed_to_ego_csv(os.path.join(out_dir, "ego_future.csv"), speed_eps=speed_eps)
    update_summary_heading_source(os.path.join(out_dir, "summary.json"), source="B: heading_vel=atan2(vy,vx)", speed_eps=speed_eps)

# 示例
# augment_export_folder(out_dir, speed_eps=0.2)


分析每个类别的数量，名称，关键帧以及是否有三条偏好轨迹。

In [52]:
import os, json
import pandas as pd

VAL_INDEX_CSV = r"/mnt/d/Datasets/WOD_E2E_Camera_v1/val_index.csv"
CLUSTER_JSON  = r"/mnt/d/Datasets/WOD_E2E_Camera_v1/meta/val_sequence_name_to_scenario_cluster.json"

df = pd.read_csv(VAL_INDEX_CSV)

with open(CLUSTER_JSON, "r", encoding="utf-8") as f:
    mapping = json.load(f)

# 你的格式：{seg_id: {"scenario_cluster": "Pedestrian"}}
seg2cls = {}
for seg_id, info in mapping.items():
    if isinstance(info, dict):
        cls = info.get("scenario_cluster")
        if cls is not None:
            seg2cls[str(seg_id)] = str(cls)

print("rows:", len(df))
print("unique seg_id:", df["seg_id"].astype(str).nunique())
print("seg2cls size:", len(seg2cls))
print("class counts (top5):\n", df["class"].value_counts().head(5))

# ---- 1) 诊断：seg_id 是否能命中 seg2cls？ ----
seg_series = df["seg_id"].astype(str)
hit = seg_series.isin(seg2cls.keys()).mean()
print(f"seg_id hit-rate vs json keys: {hit*100:.2f}%")

# 随机看一些没命中的 seg_id（如果有）
miss = seg_series[~seg_series.isin(seg2cls.keys())]
print("miss example seg_id (first 10):", miss.head(10).tolist())

# ---- 2) 覆盖 UNKNOWN（大小写都算） ----
unknown_mask = df["class"].astype(str).str.strip().str.lower().eq("unknown")
print("UNKNOWN rows:", int(unknown_mask.sum()))

# 直接 map
mapped = seg_series.map(seg2cls)  # 命中则是类别，否则 NaN

# 覆盖 UNKNOWN 且映射非空的
can_fill = unknown_mask & mapped.notna()
print("can_fill rows:", int(can_fill.sum()))

df.loc[can_fill, "class"] = mapped[can_fill]

# 覆盖后统计
print("\nAfter fill:")
print(df["class"].value_counts().head(10))
print("still UNKNOWN:", int(df["class"].astype(str).str.strip().str.lower().eq("unknown").sum()))

OUT_CSV = "/mnt/d/Datasets/WOD_E2E_Camera_v1/val_index_filled.csv"
df.to_csv(OUT_CSV, index=False)
print("saved:", OUT_CSV)


rows: 106360
unique seg_id: 479
seg2cls size: 479
class counts (top5):
 UNKNOWN    106360
Name: class, dtype: int64
seg_id hit-rate vs json keys: 100.00%
miss example seg_id (first 10): []
UNKNOWN rows: 106360
can_fill rows: 106360

After fill:
Interections             25877
Foreign Object Debris    17127
Cyclist                  15975
Pedestrian               11446
Multi-Lane Maneuvers      9317
Single-Lane Maneuvers     8499
Special Vehicles          5701
Others                    4706
Cut_ins                   4490
Construction              3222
Name: class, dtype: int64
still UNKNOWN: 0
saved: /mnt/d/Datasets/WOD_E2E_Camera_v1/val_index_filled.csv


In [56]:
import pandas as pd

PATH = "/mnt/d/Datasets/WOD_E2E_Camera_v1/val_index_filled.csv"

df = pd.read_csv(PATH)
print("Loaded:", PATH)
print("Columns:", list(df.columns))
print("Rows:", len(df))

# 1) 优先用名为 'class' 的列；如果没有，就用第一列
cls_col = "class" if "class" in df.columns else df.columns[0]
print("Using class column:", cls_col)

# 2) 清洗：转字符串 + 去空格；（可选）统一大小写
cls = df[cls_col].astype(str).str.strip()

# 如果你怀疑有大小写不一致，可以打开这一行：
# cls = cls.str.lower()

# 3) 统计
vc = cls.value_counts(dropna=False)
n_unique = int(cls.nunique(dropna=False))

print("\n=== Class stats ===")
print("Unique classes:", n_unique)

print("\nTop 30 classes by count:")
display(vc.head(30))

print("\nAll class names (first 50):")
print(list(vc.index[:50]))

# 4) 如果你想看“包含某个关键词”的类别（比如 intersection），用这个：
keyword = "inter"
hits = vc[vc.index.astype(str).str.lower().str.contains(keyword)]
print(f"\nClasses containing '{keyword}' (case-insensitive): {len(hits)}")
display(hits.head(50))


Loaded: /mnt/d/Datasets/WOD_E2E_Camera_v1/val_index_filled.csv
Columns: ['class', 'seg_id', 'key_idx', 'context_name', 'tfrecord_file', 'record_idx', 'intent', 'has_pref', 'pref_scores', 'past_len', 'future_len', 'num_images', 'cam_ids', 'front_cam_id', 'rear_cam_id', 'fps_assumed', 'time_scheme', 'output_folder']
Rows: 106360
Using class column: class

=== Class stats ===
Unique classes: 10

Top 30 classes by count:


Interections             25877
Foreign Object Debris    17127
Cyclist                  15975
Pedestrian               11446
Multi-Lane Maneuvers      9317
Single-Lane Maneuvers     8499
Special Vehicles          5701
Others                    4706
Cut_ins                   4490
Construction              3222
Name: class, dtype: int64


All class names (first 50):
['Interections', 'Foreign Object Debris', 'Cyclist', 'Pedestrian', 'Multi-Lane Maneuvers', 'Single-Lane Maneuvers', 'Special Vehicles', 'Others', 'Cut_ins', 'Construction']

Classes containing 'inter' (case-insensitive): 1


Interections    25877
Name: class, dtype: int64

In [57]:
import pandas as pd
import numpy as np

IDX = "/mnt/d/Datasets/WOD_E2E_Camera_v1/val_index_filled.csv"
df = pd.read_csv(IDX)

# ✅ 这里填你要研究的类别（注意：你的真实类别名是 'Interections'）
TARGET_CLASS = "Interections"

# --- helper: parse "9|6|10" -> [9.0, 6.0, 10.0]
def parse_scores(s):
    if pd.isna(s):
        return []
    s = str(s).strip()
    if not s:
        return []
    out = []
    for p in s.split("|"):
        p = p.strip()
        if not p:
            continue
        try:
            out.append(float(p))
        except:
            pass
    return out

# 1) filter
sub = df[df["class"].astype(str).str.strip() == TARGET_CLASS].copy()

print("TARGET_CLASS:", TARGET_CLASS)
print("rows (key-moment records):", len(sub))
print("unique seg_id:", sub["seg_id"].nunique())
print("unique (seg_id, key_idx):", sub.drop_duplicates(["seg_id","key_idx"]).shape[0])

if len(sub) == 0:
    raise ValueError(f"No rows found for class={TARGET_CLASS}. Check spelling from the 10 class names list.")

# 2) rater fields per key moment
if "pref_scores" in sub.columns:
    sub["score_list"] = sub["pref_scores"].apply(parse_scores)
else:
    sub["score_list"] = [[]]*len(sub)

sub["has_rater"]  = sub["score_list"].apply(lambda x: int(len(x) > 0))
sub["score_max"]  = sub["score_list"].apply(lambda x: max(x) if len(x)>0 else np.nan)
sub["score_mean"] = sub["score_list"].apply(lambda x: float(np.mean(x)) if len(x)>0 else np.nan)

# 3) per-seg summary
seg_summary = (
    sub.groupby("seg_id")
       .agg(
           class_name=("class", "first"),
           keyframes=("key_idx", "nunique"),
           key_idx_min=("key_idx", "min"),
           key_idx_max=("key_idx", "max"),
           intents=("intent", "nunique") if "intent" in sub.columns else ("seg_id","size"),
           rater_keymoments=("has_rater", "sum"),
           rater_rate=("has_rater", "mean"),
           score_max=("score_max", "max"),
           score_mean=("score_mean", "mean"),
       )
       .sort_values(["keyframes","rater_keymoments"], ascending=[False, False])
       .reset_index()
)

print("\n[Per-seg summary] top 20:")
display(seg_summary.head(20))

print("\nKeyframe count stats:")
display(seg_summary["keyframes"].describe())

# 4) attach key_idx list (optional but useful for later)
key_list = (
    sub.groupby("seg_id")["key_idx"]
       .apply(lambda s: sorted(pd.unique(s)))
       .reset_index()
       .rename(columns={"key_idx":"key_idx_list"})
)
seg_summary = seg_summary.merge(key_list, on="seg_id", how="left")

# 5) export
safe_name = TARGET_CLASS.replace(" ", "_").replace("/", "_")
OUT_SEG = f"/mnt/d/Datasets/WOD_E2E_Camera_v1/class_{safe_name}_segments.csv"
OUT_KF  = f"/mnt/d/Datasets/WOD_E2E_Camera_v1/class_{safe_name}_keyframes.csv"

seg_summary.to_csv(OUT_SEG, index=False)

# 每 key frame 一行：后续你要去 tfrecord 精确定位并做 rater 分析/导出素材，就用这个
kf_cols = ["class","seg_id","key_idx","tfrecord_file","record_idx","intent","pref_scores","has_pref","past_len","future_len","context_name"]
kf_cols = [c for c in kf_cols if c in sub.columns]
sub[kf_cols].sort_values(["seg_id","key_idx"]).to_csv(OUT_KF, index=False)

print("\nSaved:")
print(" -", OUT_SEG)
print(" -", OUT_KF)

print(f"\n✅ 类别 {TARGET_CLASS} 共 {seg_summary['seg_id'].nunique()} 个 segments")


TARGET_CLASS: Interections
rows (key-moment records): 25877
unique seg_id: 116
unique (seg_id, key_idx): 25877

[Per-seg summary] top 20:


,seg_id,class_name,keyframes,key_idx_min,key_idx_max,intents,rater_keymoments,rater_rate,score_max,score_mean
0,0f71456c67c9eeff7b8cea4e979e0393,Interections,230,8,237,1,230,1.0,8.0,-0.968116
1,25afa58a9e00d03f1d68ae20d26fd1ae,Interections,230,9,238,1,230,1.0,10.0,-0.957971
2,48938d756ec62f122598a77c004457af,Interections,230,8,237,2,230,1.0,10.0,-0.956522
3,73022731324e5f62ce985f3fd31702ae,Interections,230,8,237,1,230,1.0,10.0,-0.957971
4,74cf0a3e1a537277b1258a683de98b4f,Interections,230,9,238,2,230,1.0,10.0,-0.956522
5,dea3f9bccb4bf6f1d600b06d352eda84,Interections,230,8,237,3,230,1.0,10.0,-0.956522
6,eec5ca1a6526cc8e9c1a035fff8c0194,Interections,230,8,237,2,230,1.0,10.0,-0.963768
7,0081c482170156ee7aedf36157d4ff21,Interections,229,9,237,2,229,1.0,10.0,-0.956332
8,04b1507c7b4a33855b6f09a5b0050e70,Interections,229,9,237,1,229,1.0,8.0,-0.969432
9,06c2249c2322e0ffc01851bfb8eb8c4a,Interections,229,9,237,1,229,1.0,8.0,-0.967977



Keyframe count stats:


count    116.000000
mean     223.077586
std       11.989235
min      199.000000
25%      229.000000
50%      229.000000
75%      229.000000
max      230.000000
Name: keyframes, dtype: float64


Saved:
 - /mnt/d/Datasets/WOD_E2E_Camera_v1/class_Interections_segments.csv
 - /mnt/d/Datasets/WOD_E2E_Camera_v1/class_Interections_keyframes.csv

✅ 类别 Interections 共 116 个 segments


In [58]:
import pandas as pd

IDX = "/mnt/d/Datasets/WOD_E2E_Camera_v1/val_index_filled.csv"
TARGET_CLASS = "Interections"

df = pd.read_csv(IDX)
sub = df[df["class"].astype(str).str.strip() == TARGET_CLASS].copy()

sub["has_rater"] = sub["pref_scores"].astype(str).str.strip().ne("")

per_seg = (sub.groupby("seg_id")
             .agg(keyframes=("key_idx","nunique"),
                  rater_keyframes=("has_rater","sum"),
                  rater_rate=("has_rater","mean"))
             .sort_values(["rater_keyframes","keyframes"], ascending=False))

display(per_seg.head(20))
print("segments with >=1 rater keyframe:", int((per_seg["rater_keyframes"]>0).sum()), "/", len(per_seg))


,keyframes,rater_keyframes,rater_rate
seg_id,,,
0f71456c67c9eeff7b8cea4e979e0393,230,230,1.0
25afa58a9e00d03f1d68ae20d26fd1ae,230,230,1.0
48938d756ec62f122598a77c004457af,230,230,1.0
73022731324e5f62ce985f3fd31702ae,230,230,1.0
74cf0a3e1a537277b1258a683de98b4f,230,230,1.0
dea3f9bccb4bf6f1d600b06d352eda84,230,230,1.0
eec5ca1a6526cc8e9c1a035fff8c0194,230,230,1.0
0081c482170156ee7aedf36157d4ff21,229,229,1.0
04b1507c7b4a33855b6f09a5b0050e70,229,229,1.0


segments with >=1 rater keyframe: 116 / 116


In [59]:
import pandas as pd

IDX = "/mnt/d/Datasets/WOD_E2E_Camera_v1/val_index_filled.csv"
TARGET_CLASS = "Interections"   # 你要研究的类别

df = pd.read_csv(IDX)
sub = df[df["class"].astype(str).str.strip() == TARGET_CLASS].copy()

# 用 pref_scores 是否非空来判断是否有 rater（比 has_pref 更稳）
has_rater = sub["pref_scores"].astype(str).str.strip().ne("") if "pref_scores" in sub.columns else None

print("segments:", sub["seg_id"].nunique())
print("keyframes (unique seg_id,key_idx):", sub.drop_duplicates(["seg_id","key_idx"]).shape[0])

if has_rater is not None:
    print("keyframes with rater:", int(has_rater.sum()))
    print("rater coverage:", float(has_rater.mean()))


segments: 116
keyframes (unique seg_id,key_idx): 25877
keyframes with rater: 25877
rater coverage: 1.0


In [60]:
import pandas as pd

IDX = "/mnt/d/Datasets/WOD_E2E_Camera_v1/val_index_filled.csv"
df = pd.read_csv(IDX)

# 1) 定义 has_rater：pref_scores 非空就认为该帧有 rater
df["has_rater"] = df["pref_scores"].astype(str).str.strip().ne("")

# 2) 按 seg_id 统计：总 keyframes / 有 rater 的 keyframes / 覆盖率
per_seg = (
    df.groupby("seg_id")
      .agg(
          class_name=("class", "first"),
          keyframes=("key_idx", "nunique"),
          rater_frames=("has_rater", "sum"),
          rater_rate=("has_rater", "mean"),
          key_min=("key_idx", "min"),
          key_max=("key_idx", "max"),
      )
      .sort_values(["rater_frames", "keyframes"], ascending=False)
      .reset_index()
)

display(per_seg.head(30))

print("\nOverall:")
print("segments:", per_seg["seg_id"].nunique())
print("avg keyframes per seg:", per_seg["keyframes"].mean())
print("avg rater_frames per seg:", per_seg["rater_frames"].mean())
print("segments with >=1 rater frame:", int((per_seg["rater_frames"] > 0).sum()))

# 3) （可选）只看某个类别，比如 Interections
TARGET_CLASS = "Interections"
sub = per_seg[per_seg["class_name"].astype(str).str.strip() == TARGET_CLASS]
print(f"\nClass={TARGET_CLASS}: segments={len(sub)}")
display(sub.sort_values(["rater_frames","keyframes"], ascending=False).head(30))

# 4) 保存结果，后续你做导出/抽样会很方便
OUT = "/mnt/d/Datasets/WOD_E2E_Camera_v1/per_segment_rater_stats.csv"
per_seg.to_csv(OUT, index=False)
print("\nSaved:", OUT)


,seg_id,class_name,keyframes,rater_frames,rater_rate,key_min,key_max
0,09e7f15270ce9c3ea038d36c67f8ecc9,Multi-Lane Maneuvers,230,230,1.0,8,237
1,0f71456c67c9eeff7b8cea4e979e0393,Interections,230,230,1.0,8,237
2,0fff5ea6ebf1edce430191415bb02953,Cyclist,230,230,1.0,8,237
3,203e27fe231ca825208905171156a339,Others,230,230,1.0,9,238
4,25afa58a9e00d03f1d68ae20d26fd1ae,Interections,230,230,1.0,9,238
5,2f7a008f2b7d7a2c897db23af3a0b0a7,Special Vehicles,230,230,1.0,9,238
6,48938d756ec62f122598a77c004457af,Interections,230,230,1.0,8,237
7,4f9d8f8e37d3fbb5cf6d101aff958f27,Single-Lane Maneuvers,230,230,1.0,8,237
8,567443cab762895c5a05acc838764f96,Others,230,230,1.0,9,238
9,5f7187c1580d8cfca993d0cac63ba6b1,Foreign Object Debris,230,230,1.0,8,237



Overall:
segments: 479
avg keyframes per seg: 222.04592901878914
avg rater_frames per seg: 222.04592901878914
segments with >=1 rater frame: 479

Class=Interections: segments=116


,seg_id,class_name,keyframes,rater_frames,rater_rate,key_min,key_max
1,0f71456c67c9eeff7b8cea4e979e0393,Interections,230,230,1.0,8,237
4,25afa58a9e00d03f1d68ae20d26fd1ae,Interections,230,230,1.0,9,238
6,48938d756ec62f122598a77c004457af,Interections,230,230,1.0,8,237
10,73022731324e5f62ce985f3fd31702ae,Interections,230,230,1.0,8,237
12,74cf0a3e1a537277b1258a683de98b4f,Interections,230,230,1.0,9,238
20,dea3f9bccb4bf6f1d600b06d352eda84,Interections,230,230,1.0,8,237
21,eec5ca1a6526cc8e9c1a035fff8c0194,Interections,230,230,1.0,8,237
22,0081c482170156ee7aedf36157d4ff21,Interections,229,229,1.0,9,237
29,04b1507c7b4a33855b6f09a5b0050e70,Interections,229,229,1.0,9,237
35,06c2249c2322e0ffc01851bfb8eb8c4a,Interections,229,229,1.0,9,237



Saved: /mnt/d/Datasets/WOD_E2E_Camera_v1/per_segment_rater_stats.csv


In [61]:
import pandas as pd

IDX = "/mnt/d/Datasets/WOD_E2E_Camera_v1/val_index_filled.csv"
SEG = "09e7f15270ce9c3ea038d36c67f8ecc9"

df = pd.read_csv(IDX)
sub = df[df["seg_id"].astype(str) == SEG].copy()

# 用 (seg_id,key_idx) 去重，避免重复行影响统计
sub = sub.sort_values(["key_idx","record_idx"]).drop_duplicates(["seg_id","key_idx"], keep="first")

def parse_scores(s):
    if pd.isna(s): return []
    s = str(s).strip()
    if not s: return []
    out = []
    for p in s.split("|"):
        p = p.strip()
        if not p: continue
        try: out.append(float(p))
        except: pass
    return out

sub["score_list"] = sub["pref_scores"].apply(parse_scores)
sub["has_rater"] = sub["score_list"].apply(lambda x: len(x) > 0)
sub["n_scores"] = sub["score_list"].apply(len)

print("unique key_idx:", sub["key_idx"].nunique())
print("has_rater rate:", sub["has_rater"].mean())
print("n_scores value counts:")
print(sub["n_scores"].value_counts().sort_index())

# 看看有没有异常帧
bad = sub[(~sub["has_rater"]) | (sub["n_scores"] != 3)]
print("abnormal key_idx count:", len(bad))
display(bad[["key_idx","pref_scores","tfrecord_file","record_idx","context_name"]].head(20))


unique key_idx: 230
has_rater rate: 1.0
n_scores value counts:
3    230
Name: n_scores, dtype: int64
abnormal key_idx count: 0


,key_idx,pref_scores,tfrecord_file,record_idx,context_name


In [67]:
import pandas as pd
import numpy as np
import json

IN_CSV  = "/mnt/d/Datasets/WOD_E2E_Camera_v1/val_index_filled.csv"
OUT_CSV = "/mnt/d/Datasets/WOD_E2E_Camera_v1/class_segments.csv"
OUT_OVERVIEW = "/mnt/d/Datasets/WOD_E2E_Camera_v1/class_overview.csv"

df = pd.read_csv(IN_CSV)

# ---------- 基础清洗 ----------
df["class"] = df["class"].astype(str).str.strip()
df["seg_id"] = df["seg_id"].astype(str).str.strip()

# ---------- keyframe 去重：每个 (seg_id, key_idx) 保留最“可信”的一行 ----------
# 策略：优先 pref_scores 非空，然后 record_idx 最小
df["_pref_nonempty"] = df.get("pref_scores", "").astype(str).str.strip().ne("")

sort_cols = ["seg_id", "key_idx", "_pref_nonempty"]
ascending = [True, True, False]
if "record_idx" in df.columns:
    sort_cols.append("record_idx")
    ascending.append(True)

kf = (
    df.sort_values(sort_cols, ascending=ascending)
      .drop_duplicates(["seg_id", "key_idx"], keep="first")
      .drop(columns=["_pref_nonempty"])
      .copy()
)

# ---------- rater: 用 pref_scores 是否非空判断 ----------
kf["pref_scores"] = kf.get("pref_scores", "").astype(str)
kf["has_rater"] = kf["pref_scores"].str.strip().ne("")

# ---------- 分数解析：score_max / score_mean（按 keyframe 级别） ----------
def dumps_int_list(values):
    """把 pandas/numpy 的整数序列转成 Python int，再 json dumps。"""
    uniq_sorted = sorted(pd.unique(values))
    return json.dumps([int(x) for x in uniq_sorted], ensure_ascii=False)

def parse_scores(s: str):
    s = str(s).strip()
    if not s:
        return []
    out = []
    for p in s.split("|"):
        p = p.strip()
        if not p:
            continue
        try:
            out.append(float(p))
        except:
            pass
    return out

kf["score_list"] = kf["pref_scores"].apply(parse_scores)
kf["score_max_kf"] = kf["score_list"].apply(lambda x: max(x) if len(x) else np.nan)
kf["score_mean_kf"] = kf["score_list"].apply(lambda x: float(np.mean(x)) if len(x) else np.nan)

# ---------- (class, seg_id) 聚合：生成 class_segments ----------
agg_dict = {
    "keyframes": ("key_idx", "nunique"),
    "key_idx_min": ("key_idx", "min"),
    "key_idx_max": ("key_idx", "max"),
    "rater_keymoments": ("has_rater", "sum"),
    "rater_rate": ("has_rater", "mean"),
    "score_max": ("score_max_kf", "max"),
    "score_mean": ("score_mean_kf", "mean"),
}
if "intent" in kf.columns:
    agg_dict["intents"] = ("intent", "nunique")

seg_tbl = (
    kf.groupby(["seg_id", "class"])
      .agg(**agg_dict)
      .reset_index()
      .rename(columns={"class": "class_name"})
)

# ---------- key_idx_list：JSON 风格字符串 ----------
key_list_all = (
    kf.groupby(["seg_id", "class"])["key_idx"]
      .apply(dumps_int_list)
      .reset_index()
      .rename(columns={"class": "class_name", "key_idx": "key_idx_list"})
)

# ---------- key_idx_list_with_rater：只包含有 rater 的 key_idx ----------
kf_r = kf[kf["has_rater"]].copy()
key_list_rater = (
    kf_r.groupby(["seg_id", "class"])["key_idx"]
        .apply(dumps_int_list)
        .reset_index()
        .rename(columns={"class": "class_name", "key_idx": "key_idx_list_with_rater"})
)

seg_tbl = seg_tbl.merge(key_list_all, on=["seg_id", "class_name"], how="left")
seg_tbl = seg_tbl.merge(key_list_rater, on=["seg_id", "class_name"], how="left")

# 没有 rater 的 segment，with_rater 列补成空列表字符串
seg_tbl["key_idx_list_with_rater"] = seg_tbl["key_idx_list_with_rater"].fillna("[]")

# 排序
seg_tbl = seg_tbl.sort_values(["class_name", "keyframes", "rater_keymoments"], ascending=[True, False, False])

# 保存
seg_tbl.to_csv(OUT_CSV, index=False)
print("Saved:", OUT_CSV)
print("rows (segments):", len(seg_tbl))
print("unique classes:", seg_tbl["class_name"].nunique())

# ---------- 类别总览（可选） ----------
class_overview = (
    seg_tbl.groupby("class_name")
           .agg(
               segments=("seg_id", "nunique"),
               total_keyframes=("keyframes", "sum"),
               avg_keyframes_per_seg=("keyframes", "mean"),
               seg_with_any_rater=("rater_keymoments", lambda s: int((s > 0).sum())),
               avg_rater_rate=("rater_rate", "mean"),
               min_keyframes=("keyframes", "min"),
               max_keyframes=("keyframes", "max"),
           )
           .reset_index()
           .sort_values(["segments","total_keyframes"], ascending=False)
)

class_overview.to_csv(OUT_OVERVIEW, index=False)
print("Saved:", OUT_OVERVIEW)

display(seg_tbl.head(20))
display(class_overview)


Saved: /mnt/d/Datasets/WOD_E2E_Camera_v1/class_segments.csv
rows (segments): 479
unique classes: 10
Saved: /mnt/d/Datasets/WOD_E2E_Camera_v1/class_overview.csv


,seg_id,class_name,keyframes,key_idx_min,key_idx_max,rater_keymoments,rater_rate,score_max,score_mean,intents,key_idx_list,key_idx_list_with_rater
129,4224f4dc217ac4500cd9d47a392658dc,Construction,229,8,236,229,1.0,10.0,-0.959243,3,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...","[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."
152,5125dfd998e61449fe0c1f4243db286f,Construction,229,9,237,229,1.0,10.0,-0.956332,3,"[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20...","[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20..."
168,5bef8f2e82697e56d4151d1462088b90,Construction,229,9,237,229,1.0,10.0,-0.956332,3,"[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20...","[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20..."
288,a0e0bf43f0f877e13cb98d70eb482f33,Construction,229,8,236,229,1.0,10.0,-0.960699,2,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...","[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."
387,d1d943b7f0e8036803e262b8e0f9681c,Construction,229,8,236,229,1.0,10.0,-0.959243,2,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...","[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."
420,df066120eff5488fb7d9b5304243db77,Construction,229,8,236,229,1.0,8.0,-0.967977,1,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...","[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."
459,f4195a302974d6ba61a3daa811136c6f,Construction,229,8,236,229,1.0,10.0,-0.956332,3,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...","[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."
163,57e3f4d557f856f3b40d35329670f58f,Construction,225,10,234,225,1.0,10.0,-0.955556,2,"[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2...","[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2..."
207,6d2556548b36274db8d073ef6c240ace,Construction,200,29,228,200,1.0,8.0,-0.966667,2,"[29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4...","[29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4..."
48,14f0b68cf50bad87e90a490a70109cad,Construction,199,30,228,199,1.0,10.0,-0.954774,1,"[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 4...","[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 4..."


,class_name,segments,total_keyframes,avg_keyframes_per_seg,seg_with_any_rater,avg_rater_rate,min_keyframes,max_keyframes
4,Interections,116,25877,223.077586,116,1.0,199,230
3,Foreign Object Debris,78,17127,219.576923,78,1.0,182,230
2,Cyclist,71,15975,225.000000,71,1.0,152,230
7,Pedestrian,52,11446,220.115385,52,1.0,99,229
5,Multi-Lane Maneuvers,42,9317,221.833333,42,1.0,198,230
8,Single-Lane Maneuvers,38,8499,223.657895,38,1.0,174,230
9,Special Vehicles,25,5701,228.040000,25,1.0,200,230
6,Others,22,4706,213.909091,22,1.0,105,230
1,Cut_ins,20,4490,224.500000,20,1.0,199,229
0,Construction,15,3222,214.800000,15,1.0,199,229


In [68]:
import inspect
from waymo_open_dataset.wdl_limited.camera.ops import py_camera_model_ops

print("py_camera_model_ops fns:")
for name in dir(py_camera_model_ops):
    if "to_image" in name:
        print(" -", name)

def show_sig(fn_name):
    fn = getattr(py_camera_model_ops, fn_name, None)
    if fn is None:
        print(fn_name, "NOT FOUND")
        return
    try:
        print(fn_name, "signature:", inspect.signature(fn))
    except Exception as e:
        print(fn_name, "signature: <unavailable>", type(e).__name__, str(e)[:120])

show_sig("world_to_image")
show_sig("vehicle_to_image")


py_camera_model_ops fns:
 - world_to_image
 - world_to_image_moving_point
world_to_image signature: (extrinsic, intrinsic, metadata, camera_image_metadata, global_coordinate, return_depth=False, name=None)
vehicle_to_image NOT FOUND
